In [5]:
import numpy as np
import matplotlib.pyplot as plt

## Create the Jacobian

In [3]:
# From Prather, X value estimated from his final Jacobian values

# Initial mixing ratios (ppb)
y = np.array([1704, 100, 0.000026])  # [CH4, CO, OH]
x_ppb = 0.001  # External sink species (held constant)

# Source Terms (ppb yr-1)
S = np.array([177, 240, 1464])  # [S_CH4, S_CO, S_OH]

# Reaction rates (converted to per year)
k5 = 1.266e-4 * 3.156e7  # CH4 + OH
k6 = 5.08e-3 * 3.156e7   # CO + OH
k7 = 1062 * 3.156e7      # OH + X

# Reaction function with x included only internally
def reaction_rates(y):
    ch4, co, oh = y
    R5 = k5 * ch4 * oh
    R6 = k6 * co * oh
    R7 = k7 * oh * x_ppb  # OH loss to external X
    dCH4dt = S[0] - R5
    dCOdt = S[1] + R5 - R6
    dOHdt = S[2] - R5 - R6 - R7
    return np.array([dCH4dt, dCOdt, dOHdt])

# Base rate
base_rates = reaction_rates(y)

# Jacobian (numerical)
pert = 1  # 1 ppb
J = np.zeros((3, 3))

for i in range(3):
    y_pert = y.copy()
    y_pert[i] += pert
    pert_rates = reaction_rates(y_pert)
    J[:, i] = (pert_rates - base_rates) / pert

# Display Jacobian
species = ["CH4", "CO", "OH"]
print("Jacobian J (∂(d[species]/dt)/∂[species]) in ppb/yr per ppb:")
for i in range(3):
    for j in range(3):
        print(f"∂(d[{species[i]}]/dt)/∂[{species[j]}] = {J[i,j]:.6e}")


Jacobian J (∂(d[species]/dt)/∂[species]) in ppb/yr per ppb:
∂(d[CH4]/dt)/∂[CH4] = -1.038829e-01
∂(d[CH4]/dt)/∂[CO] = 0.000000e+00
∂(d[CH4]/dt)/∂[OH] = -6.808325e+06
∂(d[CO]/dt)/∂[CH4] = 1.038829e-01
∂(d[CO]/dt)/∂[CO] = -4.168445e+00
∂(d[CO]/dt)/∂[OH] = -9.224155e+06
∂(d[OH]/dt)/∂[CH4] = -1.038829e-01
∂(d[OH]/dt)/∂[CO] = -4.168445e+00
∂(d[OH]/dt)/∂[OH] = -5.635753e+07


In [7]:
eigenvalues, eigenvectors = np.linalg.eig(J)
T_years = -1/eigenvalues
modes_years = np.sort(T_years)[::-1]

# Print results
print("Eigenvalues:", eigenvalues)
print("Eigenvectors:\n", eigenvectors)
print("Perturbation Lifetimes:", T_years)

Eigenvalues: [-5.63575259e+07 -7.34954709e-02 -3.50402437e+00]
Eigenvectors:
 [[ 1.18381327e-01 -9.99373217e-01 -1.45978241e-01]
 [ 1.60387135e-01 -3.54001732e-02  9.89287801e-01]
 [ 9.79929502e-01  4.46047713e-09 -7.29029032e-08]]
Perturbation Lifetimes: [1.77438591e-08 1.36062806e+01 2.85386143e-01]


In [14]:
# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(J)

# Compute lifetimes (in years)
T_years = -1 / eigenvalues

# Print results
print("Eigenvalues:")
for i, val in enumerate(eigenvalues):
    print(f"  λ{i+1} = {val:.4e}")

print("\nEigenvectors (columns are eigenmodes):")
print(eigenvectors)

print("\nPerturbation lifetimes (in years):")
for i, T in enumerate(T_years):
    print(f"  Mode {i+1}: {T.real:.9f} years")

Eigenvalues:
  λ1 = -5.6358e+07
  λ2 = -7.3495e-02
  λ3 = -3.5040e+00

Eigenvectors (columns are eigenmodes):
[[ 1.18381327e-01 -9.99373217e-01 -1.45978241e-01]
 [ 1.60387135e-01 -3.54001732e-02  9.89287801e-01]
 [ 9.79929502e-01  4.46047713e-09 -7.29029032e-08]]

Perturbation lifetimes (in years):
  Mode 1: 0.000000018 years
  Mode 2: 13.606280603 years
  Mode 3: 0.285386143 years


In [15]:
diag = np.diag(J)
species = ["CH4", "CO", "OH"]

print("Negative Inverse of Diagonal of Jacobian (∂(d[species]/dt)/∂[same species]): Traditional Lifetime")
for i in range(3):
    print(f"∂(d[{species[i]}]/dt)/∂[{species[i]}] = {-1/diag[i]:.6e}")

Negative Inverse of Diagonal of Jacobian (∂(d[species]/dt)/∂[same species]): Traditional Lifetime
∂(d[CH4]/dt)/∂[CH4] = 9.626224e+00
∂(d[CO]/dt)/∂[CO] = 2.398976e-01
∂(d[OH]/dt)/∂[OH] = 1.774386e-08
